# Top K

## Imports

In [1]:
import sys
sys.path.append("../../")
sys.path.append("../")
from dlpmln import DeepLPMLN
import torch
from torch.autograd import Variable
import numpy as np
import time
from network import FC
from dataGen import KsData
import random
from IPython.display import display
from PIL import Image

We consider a simple version of the knapsack problem, where each item is associated with a
value and the task is to choose a subset of the items that maximizes the sum of the values of
the items. We assume there are 10 items with the same weight 2, and the capacity of the
knapsack is 15. For example,

            [2,7,3,5,2,3,8,2,1,5][1,2,3,4,5,6,9]

is a labeled example such that the first list specifies the values of the 10 items and the second
list is a solution that specifies the indices of the items to be put into the knapsack. Since the
capacity of the knapsack is fixed to be 15 and each item has weight 2, one can infer that the
solutions always contain 7 items.

## dprogram

In [2]:
dprogram='''
% define k 
#const k = 7.

topk(k).
nn(m(k,10), in, [t,f]) :- topk(k).

% we make a mistake if the total weight of the chosen items exceeds maxweight 
mistake :- #sum{1, I : in(k,I,t)} > k.
'''

dprogram_test='''
% define k 
#const k = 7.

topk(k).
% we make a mistake if the total weight of the chosen items exceeds maxweight 
mistake :- #sum{1, I : in(k,I,t)} > k.
'''

## Data Format

### dataset

A KsData object with 6 attributes: train_data, test_data, valid_data, train_labels,  

test_labels, valid_labels  
train_data is an numpy array of size (1800, 10). It consists of 1800 data as follows 

        [
        data,
        ...,
        data
        ]
        
where data is a vector (numpy array) of length 10. For example, the data shown below  

        [2 2 1 3 1 2 8 1 5 1]
        
defines the 10 values of the 10 items.  
train_labels is an numpy array of size (1800, 10). It consists of 1800 label as follows.  

        [
        label,
        ...,
        label
        ]

where label is a vector (numpy array) of length 24. For example, the label shown below  

        [0 0 1 0 0 0 0 0 0 0]

means that the item 2 is chosen to be put into the knapsack.  
test_data is a numpy array of size (600, 10).   
valid_data is a numpy array of size (600, 10).  
test_labels is a numpy array of size (600, 10).  
valid_labels is a numpy array of size (600, 10).  






## Create Neural Network

In [3]:
m = FC(10, *[50, 50, 50, 50, 50], 10)

nnMapping = {'m': m}

optimizer = {'m':torch.optim.Adam(m.parameters(), lr=0.001)}

Neural Network (MLP) Structure: (10, 50, 50, 50, 50, 50, 10)


## Create DeepLPMLN Object

In [4]:
dlpmlnObj = DeepLPMLN(dprogram, nnMapping, optimizer)

## Create dataList and obsList

In [5]:
dataset = KsData("data/data.txt",10)

dataList = []
obsList = []

for i, d in enumerate(dataset.train_data):
    d_tensor = Variable(torch.from_numpy(d).float(), requires_grad=False)
    dataList.append({"k": d_tensor})

    
with open("data/evidence_train.txt", 'r') as f:
    obsList = f.read().strip().strip("#evidence").split("#evidence")

## Create Test Data

In [6]:
testData = []
testObsLost = []

for d in dataset.test_data:
    d_tensor = Variable(torch.from_numpy(d).float(), requires_grad=False)
    testData.append({"k": d_tensor})
    

with open("data/evidence_test.txt", 'r') as f:
    testObsLost = f.read().strip().strip("#evidence").split("#evidence")

In [ ]:
for i in range(200):
	dlpmlnObj.learn(dataList = dataList, obsList = obsList, epoch=1, opt=True, storeSM=True)
	dlpmlnObj.testConstraint(testData, testObsLost,[dprogram_test])
    


Training for epoch 1 ...
